In [31]:
import pandas as pd
from google.cloud import bigquery
from cachetools import cached, TTLCache
import pickle

In [32]:
# Connexion à BigQuery
client = bigquery.Client(project='cdiscountwagon')

# Cache les résultats de chargement de données pour 10 minutes
cache = TTLCache(maxsize=100, ttl=600)

@cached(cache)
def load_data():
    query = """
    SELECT *
    FROM `cdiscountwagon.Datasets.Merges_Sales_avec_categ`
    """
    df = client.query(query).to_dataframe()
    return df

# Chargement des données
df = load_data()

In [33]:
with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_streamlit_ml/data/df_load_data.pickle','wb') as modelFile:
     pickle.dump(df,modelFile)

In [27]:
data = {
    'avg_price': [0], 
    'indice_avg_price': [0], 
    'impression_gs': [0], 
    'on_front': [0], 
    'day_of_week_number': [0], 
    'week_in_month': [0], 
    'Category_1_ANIMALERIE': [0], 
    'Category_1_AU-QUOTIDIEN': [0], 
    'Category_1_AUTO': [0], 
    'Category_1_BAGAGES': [0], 
    'Category_1_BRICOLAGE': [0], 
    'Category_1_CORNER': [0], 
    'Category_1_ELECTROMENAGER': [0], 
    'Category_1_HIGH-TECH': [0], 
    'Category_1_INFORMATIQUE': [0], 
    'Category_1_JARDIN': [0], 
    'Category_1_JEUX-PC-VIDEO-CONSOLE': [0], 
    'Category_1_JUNIORS': [0], 
    'Category_1_MAISON': [0], 
    'Category_1_MEUBLE': [0], 
    'Category_1_PRÊT-À-PORTER': [0], 
    'Category_1_PUÉRICULTURE': [0], 
    'Category_1_SPORT': [0], 
    'Category_1_TELEPHONIE': [0], 
    'Category_1_TV SON': [0], 
    'Category_1_VIN-CHAMPAGNE': [0]
}

# Créer le DataFrame à partir du dictionnaire

df_vide = pd.DataFrame(data)

In [37]:
sku_to_find = "WHIOWFC2C26X"
date = '2024-07-17'
prix_moyen = 298.35
indice_prix = 93
impression = 120000
on_front = 1


def get_category_for_sku(df_vide, sku):
    category = df.loc[df['product_id'] == sku, 'Category_1'].values
    if len(category) > 0:
        return category[0]
    else:
        return "SKU non trouvé"

# Utilisation de la fonction avec votre DataFrame df1 et le SKU spécifié
category_for_sku = get_category_for_sku(df, sku_to_find)


def encode_category(data, category):
    # Créer une copie du DataFrame pour éviter de modifier l'original
    encoded_data = data.copy()
    
    # Mettre 1 dans la colonne correspondant à la catégorie spécifiée et 0 dans les autres colonnes de catégorie
    for col in encoded_data.columns:
        if col.startswith('Category_1_'):
            if col == 'Category_1_' + category:
                encoded_data[col] = 1
            else:
                encoded_data[col] = 0
    
    return encoded_data

# Utilisation de la fonction avec votre DataFrame X_train et la catégorie spécifiée 'AUTO'
df_vide = encode_category(df_vide, category_for_sku)

# Supposons que tu as déjà un DataFrame df avec une colonne 'date_col' contenant les dates
def infos_date(date_str):
    date = pd.to_datetime(date_str, format='%Y-%m-%d')
    jour_semaine = date.dayofweek
    semaine_mois = (date.day - 1) // 7 + 1
    return jour_semaine, semaine_mois

# Remplacer la colonne 'day_of_week_number' par le numéro du jour de la semaine pour une date donnée
jour, semaine = infos_date(date)
df_vide['day_of_week_number'] = jour
df_vide['week_in_month'] = semaine
df_vide['avg_price'] = [prix_moyen] * len(df_vide)  # Assign a single value to each row
df_vide['indice_avg_price'] = [indice_prix] * len(df_vide)  # Assign a single value to each row
df_vide['impression_gs'] = [impression] * len(df_vide)  # Assign a single value to each row
df_vide['on_front'] = [on_front] * len(df_vide) 

df_vide

,avg_price,indice_avg_price,impression_gs,on_front,day_of_week_number,week_in_month,Category_1_ANIMALERIE,Category_1_AU-QUOTIDIEN,Category_1_AUTO,Category_1_BAGAGES,...,Category_1_JEUX-PC-VIDEO-CONSOLE,Category_1_JUNIORS,Category_1_MAISON,Category_1_MEUBLE,Category_1_PRÊT-À-PORTER,Category_1_PUÉRICULTURE,Category_1_SPORT,Category_1_TELEPHONIE,Category_1_TV SON,Category_1_VIN-CHAMPAGNE
0,298.35,93,120000,1,2,3,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [41]:
import dash
import dash_bootstrap_components as dbc
from dash import html

# Initialiser l'application Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Définir le layout de l'application
app.layout = html.Div([
    dbc.NavbarSimple(
        brand="Mon Application",
        brand_href="#",
        color="primary",
        dark=True,
    ),
    dbc.Container([
        dbc.Row([
            dbc.Col(html.H1("Bienvenue dans l'application Dash"), width={"size": 6, "offset": 3}),
        ]),
        dbc.Row([
            dbc.Col(dbc.Button("Cliquez-moi", color="primary"), width={"size": 4, "offset": 4}),
        ]),
    ], fluid=True)
])
